In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import keras
from keras.layers import Activation, Dense, Input
from keras.layers import Conv2D, Flatten
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.optimizers import RMSprop
from keras.models import Model
from keras.models import load_model
from keras.layers.merge import concatenate
from keras.utils import multi_gpu_model
from keras.models import load_model


from keras_contrib.layers.normalization.instancenormalization import InstanceNormalization
from keras.utils import plot_model

import numpy as np
import argparse
import datetime
from glob import glob
import os
import cv2

import librosa
import librosa.display
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
def encoder_layer(inputs,
                  filters=16,
                  kernel_size=(9,1),
                  strides=(2,1),
                  activation='relu',
                  instance_norm=True):
    """Builds a generic encoder layer made of Conv2D-IN-LeakyReLU
    IN is optional, LeakyReLU may be replaced by ReLU

    """

    conv = Conv2D(filters=filters,
                  kernel_size=kernel_size,
                  strides=strides,
                  padding='same')

    x = inputs
    if instance_norm:
        x = InstanceNormalization()(x)
    if activation == 'relu':
        x = Activation('relu')(x)
    else:
        x = LeakyReLU(alpha=0.2)(x)
    x = conv(x)

    return x


In [3]:
def decoder_layer(inputs,
                  paired_inputs,
                  filters=16,
                  kernel_size=(9,1),
                  strides=(2,1),
                  activation='relu',
                  instance_norm=True):
    """Builds a generic decoder layer made of Conv2D-IN-LeakyReLU
    IN is optional, LeakyReLU may be replaced by ReLU
    Arguments: (partial)
    inputs (tensor): the decoder layer input
    paired_inputs (tensor): the encoder layer output
          provided by U-Net skip connection &
          concatenated to inputs.

    """

    conv = Conv2DTranspose(filters=filters,
                           kernel_size=kernel_size,
                           strides=strides,
                           padding='same')

    x = inputs
    if instance_norm:
        x = InstanceNormalization()(x)
    if activation == 'relu':
        x = Activation('relu')(x)
    else:
        x = LeakyReLU(alpha=0.2)(x)
    x = conv(x)
    x = concatenate([x, paired_inputs])

    return x

In [4]:
def build_generator(input_shape,
                    output_shape=None,
                    kernel_size=(9,1),
                    name=None):
    """The generator is a U-Network made of a 4-layer encoder
    and a 4-layer decoder. Layer n-i is connected to layer i.

    Arguments:
    input_shape (tuple): input shape
    output_shape (tuple): output shape
    kernel_size (int): kernel size of encoder & decoder layers
    name (string): name assigned to generator model

    Returns:
    generator (Model):

    """

    inputs = Input(shape=input_shape)
    channels = int(output_shape[-1])
    e1 = encoder_layer(inputs,
                       32,
                       kernel_size=kernel_size,
                       activation='leaky_relu',
                       strides=1)
    e2 = encoder_layer(e1,
                       64,
                       activation='leaky_relu',
                       kernel_size=kernel_size)
    e3 = encoder_layer(e2,
                       128,
                       activation='leaky_relu',
                       kernel_size=kernel_size)
    e4 = encoder_layer(e3,
                       256,
                       activation='leaky_relu',
                       kernel_size=kernel_size)

    d1 = decoder_layer(e4,
                       e3,
                       128,
                       kernel_size=kernel_size)
    d2 = decoder_layer(d1,
                       e2,
                       64,
                       kernel_size=kernel_size)
    d3 = decoder_layer(d2,
                       e1,
                       32,
                       kernel_size=kernel_size)
    outputs = Conv2DTranspose(channels,
                              kernel_size=kernel_size,
                              strides=1,
                              activation='sigmoid',
                              padding='same')(d3)

    generator = Model(inputs, outputs, name=name)
    generator=keras.utils.multi_gpu_model(generator, gpus=2)

    return generator


In [5]:
def build_discriminator(input_shape,
                        kernel_size=(9,1),
                        patchgan=True,
                        name=None):
    """The discriminator is a 4-layer encoder that outputs either
    a 1-dim or a n x n-dim patch of probability that input is real

    Arguments:
    input_shape (tuple): input shape
    kernel_size (int): kernel size of decoder layers
    patchgan (bool): whether the output is a patch or just a 1-dim
    name (string): name assigned to discriminator model

    Returns:
    discriminator (Model):

    """

    inputs = Input(shape=input_shape)
    x = encoder_layer(inputs,
                      32,
                      kernel_size=kernel_size,
                      activation='leaky_relu',
                      instance_norm=False)
    x = encoder_layer(x,
                      64,
                      kernel_size=kernel_size,
                      activation='leaky_relu',
                      instance_norm=False)
    x = encoder_layer(x,
                      128,
                      kernel_size=kernel_size,
                      activation='leaky_relu',
                      instance_norm=False)
    x = encoder_layer(x,
                      256,
                      kernel_size=kernel_size,
                      strides=(1,1),
                      activation='leaky_relu',
                      instance_norm=False)

    # if patchgan=True use nxn-dim output of probability
    # else use 1-dim output of probability
    if patchgan:
        x = LeakyReLU(alpha=0.2)(x)
        outputs = Conv2D(1,
                         kernel_size=kernel_size,
                         strides=(2,1),
                         padding='same')(x)
    else:
        x = Flatten()(x)
        x = Dense(1)(x)
        outputs = Activation('linear')(x)


    discriminator = Model(inputs, outputs, name=name)
    discriminator = keras.utils.multi_gpu_model(discriminator, gpus=2)

    return discriminator

In [6]:
def train_cyclegan(models, data, params, test_params, test_generator):
    """ Trains the CycleGAN.

    1) Train the target discriminator
    2) Train the source discriminator
    3) Train the forward and backward cyles of adversarial networks

    Arguments:
    models (Models): Source/Target Discriminator/Generator,
                     Adversarial Model
    data (tuple): source and target training data
    params (tuple): network parameters
    test_params (tuple): test parameters
    test_generator (function): used for generating predicted target
                    and source images
    """
    # the models
    g_source, g_target, d_source, d_target, adv = models
    # network parameters
    batch_size, train_steps, patch, model_name = params
    # train dataset
    source_data, target_data, test_source_data, test_target_data = data

    titles, dirs = test_params
    dir_pred_source, dir_pred_target = dirs
    # the generator image is saved every 2000 steps
    
    save_interval = 2000
    length = min(len(test_source_data), len(test_target_data))
    test_indexes = np.random.randint(0, length, size=16)
    target_size = target_data.shape[0]
    source_size = source_data.shape[0]

    samples = test_source_data[test_indexes]
    stftdisplay(samples,
            filename='test_source_data',
            samples_dir=dir_pred_source,
            title='test_source',
            show=False)
    samples = test_target_data[test_indexes]
    stftdisplay(samples,
            filename='test_target_data',
            samples_dir=dir_pred_target,
            title='test_target',
            show=False)
    # whether to use patchgan or not
    if patch > 1:
        d_patch = (patch, patch, 1)
        valid = np.ones((batch_size,) + d_patch)
        fake = np.zeros((batch_size,) + d_patch)
    else:
        valid = np.ones([batch_size, 1])
        fake = np.zeros([batch_size, 1])

    valid_fake = np.concatenate((valid, fake))
    start_time = datetime.datetime.now()

    for step in range(train_steps):
        # sample a batch of real target data
        rand_indexes = np.random.randint(0, target_size, size=batch_size)
        real_target = target_data[rand_indexes]

        # sample a batch of real source data
        rand_indexes = np.random.randint(0, source_size, size=batch_size)
        real_source = source_data[rand_indexes]
        # generate a batch of fake target data fr real source data
        fake_target = g_target.predict(real_source)

        # combine real and fake into one batch
        x = np.concatenate((real_target, fake_target))
        # train the target discriminator using fake/real data
        metrics = d_target.train_on_batch(x, valid_fake)
        log = "%d/%d: [d_target loss: %f]" % (step+1, train_steps, metrics[0])

        # generate a batch of fake source data fr real target data
        fake_source = g_source.predict(real_target)
        x = np.concatenate((real_source, fake_source))
        # train the source discriminator using fake/real data
        metrics = d_source.train_on_batch(x, valid_fake)
        log = "%s [d_source loss: %f]" % (log, metrics[0])

        # train the adversarial network using forward and backward
        # cycles. the generated fake source and target data attempts
        # to trick the discriminators
        x = [real_source, real_target]
        y = [valid, valid, real_source, real_target]
        metrics = adv.train_on_batch(x, y)
        elapsed_time = datetime.datetime.now() - start_time
        fmt = "%s [adv loss: %f] [time: %s]"
        log = fmt % (log, metrics[0], elapsed_time)
        print(log)
        if (step + 1) % save_interval == 0:
            if (step + 1) == train_steps:
                show = True
            else:
                show = False

            test_generator((g_source, g_target),
                           (test_source_data, test_target_data),
                           step=step+1,
                           titles=titles,
                           dirs=dirs,
                           test_indexes=test_indexes,
                           show=show)

            # save the models after training the generators
            g_source.save("models/"+str(step+1) + "steps_" + model_name + "-g_source.h5")
            g_target.save("models/"+str(step+1) + "steps_" + model_name + "-g_target.h5")

In [7]:
def test_generator(generators, test_data, step,
                   titles, dirs, test_indexes=range(16),
                   show=False):
    """Test the generator models

    Arguments:
    generators (tuple): source and target generators
    test_data (tuple): source and target test data
    step (int): step number during training (0 during testing)
    titles (tuple): titles on the displayed image
    dirs (tuple): folders to save the outputs of testings
    todisplay (int): number of images to display (must be
        perfect square)
    show (bool): whether to display the image or not
          (False during training, True during testing)

    """


    # predict the output from test data
    g_source, g_target = generators
    test_source_data, test_target_data = test_data
    t1, t2, t3, t4 = titles
    title_pred_source = t1
    title_pred_target = t2
    title_reco_source = t3
    title_reco_target = t4
    dir_pred_source, dir_pred_target = dirs

    pred_target_data = g_target.predict(test_source_data)
    pred_source_data = g_source.predict(test_target_data)
    reco_source_data = g_source.predict(pred_target_data)
    reco_target_data = g_target.predict(pred_source_data)


    # display the 1st todisplay images
    samples = pred_target_data[test_indexes]
    # print(len(samples))
    # print(np.shape(samples))
    # print(samples[0])
    filename = '%06d.png' % step
    step = "Step{:,}".format(step)
    title = title_pred_target + step
    stftdisplay(samples,
            filename=filename,
            samples_dir=dir_pred_target,
            title=title,
            show=show)

    samples = pred_source_data[test_indexes]
    title = title_pred_source + step
    stftdisplay(samples,
            filename=filename,
            samples_dir=dir_pred_source,
            title=title,
            show=show)

    samples = reco_source_data[test_indexes]
    title = title_reco_source
    filename = "reconstructed_source.png"
    stftdisplay(samples,
            filename=filename,
            samples_dir=dir_pred_source,
            title=title,
            show=show)

    samples = reco_target_data[test_indexes]
    title = title_reco_target
    filename = "reconstructed_target.png"
    stftdisplay(samples,
            filename=filename,
            samples_dir=dir_pred_target,
            title=title,
            show=show)

In [8]:
def wav_spec(samples,
            filename,
            title='',
            samples_dir=None,
            show=False):

    sr = 2000
    sample_length = 8000

    # create saved_images folder
    if samples_dir is None:
        samples_dir = 'saved_images'
    save_dir = os.path.join(os.getcwd(), samples_dir, title)
    if not os.path.isdir(save_dir):
        os.makedirs(save_dir)
    filename = os.path.join(samples_dir, filename)

    fig , axes = plt.subplots(4,4)
    fig.suptitle(title)
    fig.set_size_inches(18, 9)
    for sample, ax, i in zip(samples, axes.flat, range(16)):
        sample.resize(sample_length)
        audiofile = '%s/%s-%d.wav' % (save_dir,title, i)
        librosa.output.write_wav(audiofile, sample, sr)
        spec = librosa.stft(sample)
        ax.xaxis.set_visible(False)
        ax.yaxis.set_visible(False)
        db = librosa.amplitude_to_db(np.abs(spec), ref=np.max)
        librosa.display.specshow(db, sr=sr, ax=ax, x_axis='time', y_axis='linear')

    plt.savefig(filename)
    if show:
        plt.show()

    plt.close('all')

In [9]:
def stftdisplay(samples, filename, title='',
            samples_dir=None, show=False):
    if samples_dir is None:
        samples_dir = 'saved_images'
    save_dir = os.path.join(os.getcwd(), samples_dir, title)
    if not os.path.isdir(save_dir):
        os.makedirs(save_dir)
    
    # Saving each spectogram
    for i in range(len(samples)):
        filename = os.path.join(save_dir,'%02d.png'%i)
        cv2.imwrite(filename,samples[i]*255)
    
    # filename = os.path.join(samples_dir, filename)

    # fig , axes = plt.subplots(4,4)
    # fig.suptitle(title)
    # fig.set_size_inches(18, 9)

    # for sample, ax, i in zip(samples, axes.flat, range(16)):
    #     ax.xaxis.set_visible(False)
    #     ax.yaxis.set_visible(False)
    #     ax.imshow(sample)

    # plt.savefig(filename)
    # if show:
    #     plt.show()

    # plt.close('all')

In [10]:
def build_cyclegan(shapes,
                   source_name='source',
                   target_name='target',
                   kernel_size=3,
                   patchgan=False,
                   identity=False
                   ):
    """Build the CycleGAN

    1) Build target and source discriminators
    2) Build target and source generators
    3) Build the adversarial network

    Arguments:
    shapes (tuple): source and target shapes
    source_name (string): string to be appended on dis/gen models
    target_name (string): string to be appended on dis/gen models
    kernel_size (int): kernel size for the encoder/decoder or dis/gen
                       models
    patchgan (bool): whether to use patchgan on discriminator
    identity (bool): whether to use identity loss

    Returns:
    (list): 2 generator, 2 discriminator, and 1 adversarial models

    """

    source_shape, target_shape = shapes
    lr = 2e-4
    decay = 6e-8
    gt_name = "gen_" + target_name
    gs_name = "gen_" + source_name
    dt_name = "dis_" + target_name
    ds_name = "dis_" + source_name

    # build target and source generators
    g_target = build_generator(source_shape,
                               target_shape,
                               kernel_size=kernel_size,
                               name=gt_name)
    g_source = build_generator(target_shape,
                               source_shape,
                               kernel_size=kernel_size,
                               name=gs_name)
    print('---- TARGET GENERATOR ----')
    g_target.summary()
    print('---- SOURCE GENERATOR ----')
    g_source.summary()

    # build target and source discriminators
    d_target = build_discriminator(target_shape,
                                   patchgan=patchgan,
                                   kernel_size=kernel_size,
                                   name=dt_name)
    d_source = build_discriminator(source_shape,
                                   patchgan=patchgan,
                                   kernel_size=kernel_size,
                                   name=ds_name)
    print('---- TARGET DISCRIMINATOR ----')
    d_target.summary()
    print('---- SOURCE DISCRIMINATOR ----')
    d_source.summary()

    optimizer = RMSprop(lr=lr, decay=decay)
    d_target.compile(loss='mse',
                     optimizer=optimizer,
                     metrics=['accuracy'])
    d_source.compile(loss='mse',
                     optimizer=optimizer,
                     metrics=['accuracy'])

    d_target.trainable = False
    d_source.trainable = False

    # build the computational graph for the adversarial model
    # forward cycle network and target discriminator
    source_input = Input(shape=source_shape)
    fake_target = g_target(source_input)
    preal_target = d_target(fake_target)
    reco_source = g_source(fake_target)

    # backward cycle network and source discriminator
    target_input = Input(shape=target_shape)
    fake_source = g_source(target_input)
    preal_source = d_source(fake_source)
    reco_target = g_target(fake_source)

    # if we use identity loss, add 2 extra loss terms
    # and outputs
    if identity:
        iden_source = g_source(source_input)
        iden_target = g_target(target_input)
        loss = ['mse', 'mse', 'mae', 'mae', 'mae', 'mae']
        loss_weights = [1., 1., 10., 10., 0.5, 0.5]
        inputs = [source_input, target_input]
        outputs = [preal_source,
                   preal_target,
                   reco_source,
                   reco_target,
                   iden_source,
                   iden_target]
    else:
        loss = ['mse', 'mse', 'mae', 'mae']
        loss_weights = [1., 1., 10., 10.]
        inputs = [source_input, target_input]
        outputs = [preal_source,
                   preal_target,
                   reco_source,
                   reco_target]

    # build adversarial model
    adv = Model(inputs, outputs, name='adversarial')
    adv=keras.utils.multi_gpu_model(adv, gpus=2)
    optimizer = RMSprop(lr=lr*0.5, decay=decay*0.5)
    adv.compile(loss=loss,
                loss_weights=loss_weights,
                optimizer=optimizer,
                metrics=['accuracy'])
    print('---- ADVERSARIAL NETWORK ----')
    adv.summary()

    # os.makedirs('models/base-cgan', exist_ok=True)
    # plot_model(g_source, to_file='models/base-cgan/g_source.png', show_shapes=True)
    # plot_model(g_target, to_file='models/base-cgan/g_target.png', show_shapes=True)
    # plot_model(d_source, to_file='models/base-cgan/d_source.png', show_shapes=True)
    # plot_model(d_target, to_file='models/base-cgan/d_target.png', show_shapes=True)
    # plot_model(adv, to_file='models/base-cgan/adv.png', show_shapes=True)
    return g_source, g_target, d_source, d_target, adv

In [11]:

def load_data(source_name='Piano',
            target_name='Guitar',
            sample_length=8000
            ):
    # data = {}
    # domain_cats = []
    # for domain in ['valid', 'test']:
    #     path = './data/nsynth-processed-%s/' % domain
    #     for npy_file in sorted(glob(os.path.join(path, '*.npy'))):
    #         cat = npy_file[len(path) + len(domain) + 1: -4]
    #         if cat in [source_name, target_name]:
    #             domain_cat = '%s-%s' % (domain,cat)
    #             domain_cats.append(domain_cat)
    #             data[domain_cat] = []
    #             data[domain_cat] = np.load(npy_file)
    #             print(npy_file[len(path) : ], np.shape(data[domain_cat]))
    # for domain in ['test','train']:
    #     path = './stft_dir/%s' %domain
    #     for imgs in sorted(glob(os.path.join(path, '*.png'))):
    #         cat = imgs[len(path) + len(domain) + 1]
    #         if cat in [source_name, target_name]:
    source_data, test_source_data, target_data, test_target_data = [], [], [], []
    for imgs in sorted(glob('./stft_data/Piano/train/*.png')):
        imgA = cv2.imread(imgs,1).astype('float32')/255
        source_data.append(np.asarray(imgA))
    print(np.shape(source_data))

    for imgs in sorted(glob('./stft_data/Piano/test/*.png')):
        imgA = cv2.imread(imgs,1).astype('float32')/255
        test_source_data.append(np.asarray(imgA))
    print(np.shape(test_source_data))

    for imgs in sorted(glob('./stft_data/Guitar/train/*.png')):
        imgA = cv2.imread(imgs,1).astype('float32')/255
        target_data.append(np.asarray(imgA))
    print(np.shape(target_data))

    for imgs in sorted(glob('./stft_data/Guitar/test/*.png')):
        imgA = cv2.imread(imgs,1).astype('float32')/255
        test_target_data.append(np.asarray(imgA))
    print(np.shape(test_target_data))
             
    # source_data = np.reshape(data[domain_cats[0]], (len(data[domain_cats[0]]),sample_length,1,1))
    # target_data = np.reshape(data[domain_cats[1]], (len(data[domain_cats[1]]),sample_length,1,1))
    # test_source_data = np.reshape(data[domain_cats[2]], (len(data[domain_cats[2]]),sample_length,1,1))
    # test_target_data = np.reshape(data[domain_cats[3]], (len(data[domain_cats[3]]),sample_length,1,1))

    data = (source_data, target_data, test_source_data, test_target_data)
    return data

In [12]:
def piano2guitar(g_models=None):
    """
    # keyboard_electronic <--> guitar_acoustic
    piano_acoustic <-> guitar_acoustic
    """
    cats = [
            'bass_electronic', 'bass_synthetic', 'brass_acoustic', 'flute_acoustic',
            'flute_synthetic', 'guitar_acoustic', 'guitar_electronic',
            'keyboard_acoustic', 'keyboard_electronic', 'keyboard_synthetic',
            'mallet_acoustic', 'organ_electronic', 'reed_acoustic', 'string_acoustic',
            'vocal_acoustic', 'vocal_synthetic'
            ]

    sample_length = 8000

    source_shape = (480, 640, 3)
    target_shape = (480, 640, 3)

   
    shapes = (source_shape, target_shape)
    kernel_size = (9,1)
    source_name = 'piano_acoustic'
    target_name = 'guitar_acoustic'

    source_data = np.load('./stft_dir/%s-train.npy' % source_name)
    target_data = np.load('./stft_dir/%s-train.npy' % target_name)
    test_source_data = np.load('./stft_dir/%s-test.npy' % source_name)
    test_target_data = np.load('./stft_dir/%s-test.npy' % target_name)

    print(np.shape(source_data), np.shape(target_data), np.shape(test_source_data), np.shape(test_target_data))
    data = (source_data, target_data, test_source_data, test_target_data)
    model_name = 'cyclegan_ka_ga'
    batch_size = 1
    train_steps = 20000
    patchgan = False

    # patch size is divided by 2^n since we downscaled the input
    # in the discriminator by 2^n (ie. we use strides=2 n times)
    patch = int(source_data.shape[1] / 2**4) if patchgan else 1

    postfix = ('9x1p') if patchgan else ('9x1')
    titles = ('%s_predicted-source' % source_name,
             '%s_predicted-target' % target_name,
             '%s_reconstructed-source' % source_name,
             '%s_reconstructed-target' % target_name)
    dirs = ('%s_source-%s' % (source_name,postfix), '%s_target-%s' % (target_name,postfix))

    params = (batch_size, train_steps, patch, model_name)
    test_params = (titles, dirs)

    models = build_cyclegan(shapes,
                       source_name=source_name,
                       target_name=target_name,
                       kernel_size=kernel_size,
                       patchgan=patchgan
                       )

    train_cyclegan(models,
                data,
                params,
                test_params,
                test_generator)

if __name__ == '__main__':
    piano2guitar()

(785, 480, 640, 3) (1000, 480, 640, 3) (119, 480, 640, 3) (481, 480, 640, 3)
---- TARGET GENERATOR ----
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 480, 640, 3)  0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 480, 640, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
lambda_2 (Lambda)               (None, 480, 640, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
gen_guitar_acoustic (Model)     (None, 480, 640, 3)  869617      lambda_1[0][0]                   
     

/home/samsungpc/anaconda3/envs/tfenv/lib/python3.6/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


1/20000: [d_target loss: 0.502729] [d_source loss: 0.485208] [adv loss: 55430.855469] [time: 0:00:30.137628]
2/20000: [d_target loss: 32298.113281] [d_source loss: 24798.335938] [adv loss: 8409.562500] [time: 0:00:31.463367]
3/20000: [d_target loss: 2330.557617] [d_source loss: 4981.446289] [adv loss: 52.771938] [time: 0:00:32.781049]
4/20000: [d_target loss: 44.474899] [d_source loss: 92.431221] [adv loss: 48.819458] [time: 0:00:34.100915]
5/20000: [d_target loss: 17.891216] [d_source loss: 20.973009] [adv loss: 37.761238] [time: 0:00:35.425096]
6/20000: [d_target loss: 8.385096] [d_source loss: 7.241947] [adv loss: 27.614491] [time: 0:00:36.745539]
7/20000: [d_target loss: 3.874062] [d_source loss: 3.059087] [adv loss: 20.576305] [time: 0:00:38.072025]
8/20000: [d_target loss: 1.672207] [d_source loss: 1.374304] [adv loss: 16.140387] [time: 0:00:39.386135]
9/20000: [d_target loss: 0.670348] [d_source loss: 0.607520] [adv loss: 13.526217] [time: 0:00:40.706618]
10/20000: [d_target los

78/20000: [d_target loss: 0.250991] [d_source loss: 8.199211] [adv loss: 26.519003] [time: 0:02:12.180410]
79/20000: [d_target loss: 1.079433] [d_source loss: 8.970815] [adv loss: 16.751499] [time: 0:02:13.512380]
80/20000: [d_target loss: 5.065686] [d_source loss: 9.017695] [adv loss: 9.871526] [time: 0:02:14.843862]
81/20000: [d_target loss: 24.556564] [d_source loss: 24.189808] [adv loss: 14.066582] [time: 0:02:16.178502]
82/20000: [d_target loss: 92.991989] [d_source loss: 36.058018] [adv loss: 19.394108] [time: 0:02:17.513294]
83/20000: [d_target loss: 137.372681] [d_source loss: 32.837143] [adv loss: 9.717358] [time: 0:02:18.844180]
84/20000: [d_target loss: 35.088310] [d_source loss: 24.874596] [adv loss: 21.383965] [time: 0:02:20.175336]
85/20000: [d_target loss: 24.012508] [d_source loss: 37.679081] [adv loss: 14.899870] [time: 0:02:21.507972]
86/20000: [d_target loss: 17.266827] [d_source loss: 32.569027] [adv loss: 31.367290] [time: 0:02:22.837424]
87/20000: [d_target loss: 

155/20000: [d_target loss: 3.157263] [d_source loss: 0.023353] [adv loss: 8.438927] [time: 0:03:54.709693]
156/20000: [d_target loss: 1.373473] [d_source loss: 0.082142] [adv loss: 7.405565] [time: 0:03:56.038080]
157/20000: [d_target loss: 0.621737] [d_source loss: 0.438127] [adv loss: 17.501389] [time: 0:03:57.371894]
158/20000: [d_target loss: 0.174960] [d_source loss: 0.898525] [adv loss: 6.740520] [time: 0:03:58.704872]
159/20000: [d_target loss: 0.042929] [d_source loss: 0.423019] [adv loss: 8.729431] [time: 0:04:00.039145]
160/20000: [d_target loss: 0.019559] [d_source loss: 0.196646] [adv loss: 7.900248] [time: 0:04:01.372272]
161/20000: [d_target loss: 0.001088] [d_source loss: 0.438243] [adv loss: 10.411580] [time: 0:04:02.702133]
162/20000: [d_target loss: 0.021535] [d_source loss: 1.506918] [adv loss: 8.150727] [time: 0:04:04.037947]
163/20000: [d_target loss: 0.015738] [d_source loss: 4.272645] [adv loss: 8.214725] [time: 0:04:05.369845]
164/20000: [d_target loss: 0.053616

...

19622/20000: [d_target loss: 0.990554] [d_source loss: 0.343717] [adv loss: 0.625228] [time: 7:24:09.213598]
19623/20000: [d_target loss: 0.200000] [d_source loss: 0.421949] [adv loss: 21.013552] [time: 7:24:10.545938]
19624/20000: [d_target loss: 0.115509] [d_source loss: 10.946836] [adv loss: 1.112789] [time: 7:24:11.881023]
19625/20000: [d_target loss: 0.051494] [d_source loss: 1.837756] [adv loss: 34.469894] [time: 7:24:13.213798]
19626/20000: [d_target loss: 0.046711] [d_source loss: 19.969509] [adv loss: 0.805754] [time: 7:24:14.559383]
19627/20000: [d_target loss: 0.284217] [d_source loss: 0.090368] [adv loss: 1.866257] [time: 7:24:15.892202]
19628/20000: [d_target loss: 0.635708] [d_source loss: 0.031516] [adv loss: 1.534001] [time: 7:24:17.228284]
19629/20000: [d_target loss: 0.267525] [d_source loss: 0.428004] [adv loss: 0.880269] [time: 7:24:18.563831]
19630/20000: [d_target loss: 0.092806] [d_source loss: 0.084621] [adv loss: 1.854432] [time: 7:24:19.896275]
19631/20000: [d

19698/20000: [d_target loss: 0.148009] [d_source loss: 0.916460] [adv loss: 0.881533] [time: 7:25:50.737011]
19699/20000: [d_target loss: 0.039654] [d_source loss: 0.207584] [adv loss: 24.284939] [time: 7:25:52.073916]
19700/20000: [d_target loss: 0.122403] [d_source loss: 15.732105] [adv loss: 0.886859] [time: 7:25:53.421921]
19701/20000: [d_target loss: 0.055489] [d_source loss: 0.257809] [adv loss: 10.223495] [time: 7:25:54.752575]
19702/20000: [d_target loss: 0.412834] [d_source loss: 3.961608] [adv loss: 0.647946] [time: 7:25:56.090877]
19703/20000: [d_target loss: 0.304717] [d_source loss: 1.224705] [adv loss: 2.344461] [time: 7:25:57.429108]
19704/20000: [d_target loss: 0.189150] [d_source loss: 0.116800] [adv loss: 1.751439] [time: 7:25:58.760264]
19705/20000: [d_target loss: 0.019836] [d_source loss: 0.252247] [adv loss: 1.706630] [time: 7:26:00.092632]
19706/20000: [d_target loss: 0.723703] [d_source loss: 0.140232] [adv loss: 0.655390] [time: 7:26:01.410027]
19707/20000: [d_

19774/20000: [d_target loss: 0.015049] [d_source loss: 0.131829] [adv loss: 1.632453] [time: 7:27:32.324555]
19775/20000: [d_target loss: 0.160525] [d_source loss: 0.191635] [adv loss: 1.599416] [time: 7:27:33.654350]
19776/20000: [d_target loss: 0.339339] [d_source loss: 0.239436] [adv loss: 1.638584] [time: 7:27:34.985062]
19777/20000: [d_target loss: 0.143507] [d_source loss: 0.063371] [adv loss: 1.609290] [time: 7:27:36.327575]
19778/20000: [d_target loss: 0.095147] [d_source loss: 0.080402] [adv loss: 1.591853] [time: 7:27:37.650961]
19779/20000: [d_target loss: 0.162339] [d_source loss: 0.222942] [adv loss: 1.119106] [time: 7:27:38.988202]
19780/20000: [d_target loss: 0.158548] [d_source loss: 0.081084] [adv loss: 2.087563] [time: 7:27:40.323191]
19781/20000: [d_target loss: 0.039791] [d_source loss: 0.127082] [adv loss: 1.396216] [time: 7:27:41.655829]
19782/20000: [d_target loss: 0.045817] [d_source loss: 0.018385] [adv loss: 2.221217] [time: 7:27:42.986821]
19783/20000: [d_tar

19850/20000: [d_target loss: 0.246548] [d_source loss: 8.489074] [adv loss: 2.926037] [time: 7:29:13.835299]
19851/20000: [d_target loss: 0.322633] [d_source loss: 0.652296] [adv loss: 9.006893] [time: 7:29:15.168020]
19852/20000: [d_target loss: 0.206045] [d_source loss: 10.012121] [adv loss: 1.463704] [time: 7:29:16.507980]
19853/20000: [d_target loss: 0.120047] [d_source loss: 0.049905] [adv loss: 1.393654] [time: 7:29:17.837050]
19854/20000: [d_target loss: 0.128546] [d_source loss: 0.070197] [adv loss: 1.483077] [time: 7:29:19.166133]
19855/20000: [d_target loss: 0.265653] [d_source loss: 0.468917] [adv loss: 6.060156] [time: 7:29:20.505730]
19856/20000: [d_target loss: 0.528292] [d_source loss: 0.229823] [adv loss: 1.421900] [time: 7:29:21.839057]
19857/20000: [d_target loss: 0.193313] [d_source loss: 0.061541] [adv loss: 1.077531] [time: 7:29:23.175923]
19858/20000: [d_target loss: 0.032098] [d_source loss: 0.298393] [adv loss: 1.237874] [time: 7:29:24.517545]
19859/20000: [d_ta

19926/20000: [d_target loss: 0.230991] [d_source loss: 0.099473] [adv loss: 0.995075] [time: 7:30:55.315732]
19927/20000: [d_target loss: 0.234288] [d_source loss: 0.082832] [adv loss: 1.568569] [time: 7:30:56.660400]
19928/20000: [d_target loss: 0.176362] [d_source loss: 0.218114] [adv loss: 1.256993] [time: 7:30:57.989631]
19929/20000: [d_target loss: 0.075472] [d_source loss: 0.168144] [adv loss: 3.565163] [time: 7:30:59.326860]
19930/20000: [d_target loss: 0.667554] [d_source loss: 0.339256] [adv loss: 0.680273] [time: 7:31:00.661360]
19931/20000: [d_target loss: 0.254700] [d_source loss: 0.147760] [adv loss: 5.232670] [time: 7:31:01.999616]
19932/20000: [d_target loss: 1.079098] [d_source loss: 0.496279] [adv loss: 0.549494] [time: 7:31:03.329468]
19933/20000: [d_target loss: 0.090407] [d_source loss: 0.134995] [adv loss: 10.711823] [time: 7:31:04.659794]
19934/20000: [d_target loss: 3.539953] [d_source loss: 0.136375] [adv loss: 0.191929] [time: 7:31:05.993772]
19935/20000: [d_ta